In [1]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

In [2]:
raw = pd.read_csv('apartments.csv')
raw

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price
0,1,1,750,546.0,1,1951,4,W,Yes,175000.0
1,1,1,700,230.0,0,1895,2,W,No,199000.0
2,3,2,1600,150.0,0,2020,3,E,No,279000.0
3,1,1,985,424.0,1,1892,3,W,Yes,210000.0
4,2,2,1200,973.0,1,1965,8,E,Yes,209900.0


In [3]:
df = raw.copy()

In [4]:
df['exposure'] = df.apply(lambda r: 1 if r['exposure'] == 'E' else 0, axis=1)

In [5]:
df['elevator'] = df.apply(lambda r: 1 if r['elevator'] == 'Yes' else 0, axis=1)

In [6]:
df

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price
0,1,1,750,546.0,1,1951,4,0,1,175000.0
1,1,1,700,230.0,0,1895,2,0,0,199000.0
2,3,2,1600,150.0,0,2020,3,1,0,279000.0
3,1,1,985,424.0,1,1892,3,0,1,210000.0
4,2,2,1200,973.0,1,1965,8,1,1,209900.0


In [7]:
mms = MinMaxScaler(feature_range=(10, 100))

In [8]:
df = pd.DataFrame(mms.fit_transform(df), columns=df.columns)
df

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price
0,10.0,10.0,15.0,53.304982,100.0,51.484375,40.0,10.0,100.0,10.000000
1,10.0,10.0,10.0,18.748481,10.0,12.109375,10.0,10.0,10.0,30.769231
2,100.0,100.0,100.0,10.000000,10.0,100.000000,25.0,100.0,10.0,100.000000
3,10.0,10.0,38.5,39.963548,100.0,10.000000,25.0,10.0,100.0,40.288462
4,55.0,100.0,60.0,100.000000,100.0,61.328125,100.0,100.0,100.0,40.201923


In [9]:
weights = {'bedrooms'     :   1.,
           'bathrooms'     :  1.,
           'area'          :  1.,
           'hoa'           : -1.,
           'parking'       :  1.,
           'year'          :  1.,
           'floor'         : -1.,
           'price'         : -1.,
           'exposure'      :  1.,
           'elevator'      :  1.}

In [10]:
(10        **  1) * \
(10        **  1) * \
(15        **  1) * \
(53.304982 ** -1) * \
(100       **  1) * \
(51.484375 **  1) * \
(40        ** -1) * \
(10        ** -1) * \
(10        **  1) * \
(100       **  1)

362192.0484843237

In [11]:
def wpm(option, weights):
    value = 1
    for column in option.keys():
        try:
            value *= option[column] ** weights[column]
        except KeyError:
            pass
    return value

In [12]:
wpm(df.loc[0], weights)

362192.05001994525

In [13]:
df['wpm'] = df.apply(lambda r: wpm(r, weights), axis=1)
df.sort_values(by='wpm', ascending=False)

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price,wpm
4,55.0,100.0,60.0,100.000000,100.0,61.328125,100.0,100.0,100.0,40.201923,5.034157e+07
2,100.0,100.0,100.0,10.000000,10.0,100.000000,25.0,100.0,10.0,100.000000,4.000000e+07
0,10.0,10.0,15.0,53.304982,100.0,51.484375,40.0,10.0,100.0,10.000000,3.621921e+05
3,10.0,10.0,38.5,39.963548,100.0,10.000000,25.0,10.0,100.0,40.288462,9.564802e+04
1,10.0,10.0,10.0,18.748481,10.0,12.109375,10.0,10.0,10.0,30.769231,2.099128e+03


In [14]:
pd.merge(raw, df['wpm'], left_index=True, right_index=True) \
  .sort_values(by='wpm', ascending=False)

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price,wpm
4,2,2,1200,973.0,1,1965,8,E,Yes,209900.0,5.034157e+07
2,3,2,1600,150.0,0,2020,3,E,No,279000.0,4.000000e+07
0,1,1,750,546.0,1,1951,4,W,Yes,175000.0,3.621921e+05
3,1,1,985,424.0,1,1892,3,W,Yes,210000.0,9.564802e+04
1,1,1,700,230.0,0,1895,2,W,No,199000.0,2.099128e+03


In [15]:
weights = {'bedrooms'     :   1.,
           'bathrooms'     :  1.,
           'area'          :  3.,
           'hoa'           : -5.,
           'parking'      :   5.,
           'year'          :  1.,
           'floor'         : -5.,
           'price'         : -3.,
           'exposure_east' :  1.,
           'has_elevator'  :  3.
}

In [16]:
df['wpm'] = df.apply(lambda r: wpm(r, weights), axis=1)
df.sort_values(by='wpm', ascending=False)

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price,wpm
2,100.0,100.0,100.0,10.000000,10.0,100.000000,25.0,100.0,10.0,100.000000,0.102400
3,10.0,10.0,38.5,39.963548,100.0,10.000000,25.0,10.0,100.0,40.288462,0.008766
0,10.0,10.0,15.0,53.304982,100.0,51.484375,40.0,10.0,100.0,10.000000,0.003943
4,55.0,100.0,60.0,100.000000,100.0,61.328125,100.0,100.0,100.0,40.201923,0.000112
1,10.0,10.0,10.0,18.748481,10.0,12.109375,10.0,10.0,10.0,30.769231,0.000018


In [17]:
pd.merge(raw, df['wpm'], left_index=True, right_index=True) \
  .sort_values(by='wpm', ascending=False)

,bedrooms,bathrooms,area,hoa,parking,year,floor,exposure,elevator,price,wpm
2,3,2,1600,150.0,0,2020,3,E,No,279000.0,0.102400
3,1,1,985,424.0,1,1892,3,W,Yes,210000.0,0.008766
0,1,1,750,546.0,1,1951,4,W,Yes,175000.0,0.003943
4,2,2,1200,973.0,1,1965,8,E,Yes,209900.0,0.000112
1,1,1,700,230.0,0,1895,2,W,No,199000.0,0.000018
